# Wissensaggregator Mittelalter und frühe Neuzeit

Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
wds_path="../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise, MySQL

In [6]:
using WiagDataSetup

In [7]:
using MySQL, DataFrames, CSV

In [8]:
Wds = WiagDataSetup

WiagDataSetup

In [9]:
Wds.setDBWIAG(user="georg", db="wiag_in")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

##  Finde Orte, die in der Klosterdatenbank mit Feature-Code A statt P eingetragen sind.
*2022-12-12*

Lies Klosterstandorte der Klosterdatenbank mit Koordinaten und GeoNames ID, deren GeoNames-ID auf einen Eintrag mit Feature A statt P verweist.

In [10]:
path_elmts = (
    "C:\\",
    "Users",
    "georg",
    "Documents",
    "projekte-doc",
    "WIAGweb2",
    "data",
    "Klosterort_GeoNames_ID.csv"
)
institution_file = joinpath(path_elmts)

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\Klosterort_GeoNames_ID.csv"

In [11]:
using CSV, DataFrames

In [12]:
df_inst = CSV.read(institution_file, DataFrame);

In [13]:
size(df_inst)

(312, 3)

In [14]:
first(df_inst, 7)

Row,place_name,gsn,geonames_id
,String,Int64,Int64
1,Adelberg,20036,6555436
2,Aichtal,20023,2958914
3,Aigen-Schlägl,90834,11592639
4,Almkerk,8156,275987
5,Almkerk,8156,2750324
6,Altenmarkt a.d. Alz,60046,6559209
7,Andenne,1190,2803205


Tabelle aller Orte in GeoNames

In [15]:
using MySQL

In [16]:
table = "place_all"
sql = "SELECT id, geonames_id, name, latitude, longitude, feature_class " *
"FROM place_all " *
"WHERE feature_class = 'P'"
df_p_all = Wds.sql_df(sql);

In [17]:
size(df_p_all)

(390505, 6)

In [18]:
using HTTP, JSON

In [167]:
columns = [
    "monasteryLocationId" => String[],
    "geonamesId"          => String[],
    "countryName"         => String[],
    "diocese"             => String[],
    "placeName"           => String[],
    "latitude"            => String[],
    "longitude"           => String[],
    "locationName"        => String[],    
    "beginTPQ"            => String[],
    "endTPQ"              => String[],
    "to"                  => String[],    
    "from"                => String[],
];

In [168]:
df_gs_mon = DataFrame(columns)

0×12 DataFrame

In [29]:
query_base = "https://api.gs.sub.uni-goettingen.de/v1/monastery"

"https://api.gs.sub.uni-goettingen.de/v1/monastery"

Test für ein einzelnes Kloster

In [26]:
gsn = 3416

3416

In [30]:
query = join([query_base, gsn], "/")

"https://api.gs.sub.uni-goettingen.de/v1/monastery/3416"

In [37]:
response = HTTP.get(query);

In [38]:
mon = JSON.parse(String(response.body));

Dict{String, Any} with 18 entries:
  "orders"         => Any[Dict{String, Any}("name"=>"Benediktiner", "abbreviati…
  "persons"        => Any[]
  "personsNote"    => nothing
  "places"         => Any[]
  "locations"      => Any[Dict{String, Any}("monasteryLocationId"=>"1822", "end…
  "name"           => "Kollegiatstift St. Hadelin, Visé, Belgien"
  "altLabels"      => Any[]
  "orderTimeRange" => Any[]
  "externalUrls"   => Any[Dict{String, Any}("label"=>"Wikidata", "value"=>"Q107…
  "gnd"            => nothing
  "dioceses"       => Any[]
  "note"           => "1338 von Celles nach Visé (St. Martin) verlegt. Dort wir…
  "monasteryBooks" => Any[]
  "gsVolumes"      => Any[]
  "volumeNumber"   => Any[]
  "gsnId"          => "3416"
  "patrocinium"    => "Hadelin"
  "literature"     => Any[Dict{String, Any}("detail"=>"S. 187", "fullTitle"=>"W…

In [41]:
loc_list = mon["locations"];

In [42]:
loc = loc_list[1]

Dict{String, Any} with 12 entries:
  "monasteryLocationId" => "1822"
  "endTPQ"              => "1797"
  "geonamesId"          => "2784548"
  "diocese"             => "Lüttich"
  "placeName"           => "Visé"
  "latitude"            => "50.73316"
  "locationName"        => nothing
  "longitude"           => "5.69606"
  "beginTPQ"            => "1338"
  "to"                  => "1797"
  "countryName"         => "Belgien"
  "from"                => "1338"

In [48]:
push!(df_gs_mon, loc, promote=true)

Row,monasteryLocationId,geonamesId,countryName,diocese,placeName,latitude,longitude,locationName,beginTPQ,endTPQ,to,from
,Any,Any,String,String,String,String,String,Union…,String,String,String,String
1,1822,2784548,Belgien,Lüttich,Visé,50.73316,5.69606,,1338,1797,1797,1338


Frage für alle Klöster mit Feature Code A die Klosterdatenbank ab, um die Koordinaten auszulesen.

In [63]:
function fill_gs_mon!(df_gs_mon, df_A)
    n = 0
    for r_a in eachrow(df_A)
        query = join([query_base, r_a.gsn], "/")        
        response = HTTP.get(query)
        mon = JSON.parse(String(response.body))
        for loc in mon["locations"]
            push!(df_gs_mon, loc, promote = true)
            n += 1
            if n % 100 == 0
                @info "" n
            end
        end
    end
    return n
end

fill_gs_mon! (generic function with 1 method)

In [169]:
fill_gs_mon!(df_gs_mon, df_inst)

┌ Info: 
│   n = 100
└ @ Main In[63]:11
┌ Info: 
│   n = 200
└ @ Main In[63]:11
┌ Info: 
│   n = 300
└ @ Main In[63]:11
┌ Info: 
│   n = 400
└ @ Main In[63]:11


409

In [248]:
columns = [
    :monasteryLocationId,
    :geonamesId,
    :placeName,
    :diocese,
    :latitude,
    :longitude,    
];

In [249]:
df_cand = select(df_gs_mon, columns);

In [250]:
df_cand[51:55, [:geonamesId, :placeName, :latitude, :longitude]]

Row,geonamesId,placeName,latitude,longitude
,String,String,String,String
1,2747351,'s-Hertogenbosch,51.6789560440338,5.34561970942765
2,2758459,Boxtel,51.58305556,5.324166667
3,2747351,'s-Hertogenbosch,51.68694444,5.31
4,2758459,Boxtel,51.5891579239149,5.32559784224639
5,2758459,Boxtel,51.5891579239149,5.32559784224639


In [251]:
unique!(df_cand, :geonamesId);

In [252]:
size(df_cand)

(208, 6)

In [253]:
transform!(df_cand, :latitude => ByRow(x -> parse(Float64, x)) => :latitude);

In [254]:
transform!(df_cand, :longitude => ByRow(x -> parse(Float64, x)) => :longitude);

Finde Orte in der Nähe in Tabelle `place_all` mit `feature_code` = `P`

In [139]:
using NearestNeighbors, StaticArrays

In [189]:
tree = KDTree([SVector(d.latitude, d.longitude) for d in eachrow(df_p_all)])

KDTree{SVector{2, DecFP.Dec64}, Euclidean, DecFP.Dec64}
  Number of points: 390505
  Dimensions: 2
  Metric: Euclidean(0.0)
  Reordered: true

In [255]:
points = [SVector(d.latitude, d.longitude) for d in eachrow(df_cand)];

In [256]:
idxs, dists = nn(tree, points)

([329045, 309494, 181063, 2921, 7125, 325100, 20048, 12120, 327198, 102349  …  4014, 1367, 255958, 254734, 253634, 286818, 297682, 253222, 118, 299576], DecFP.Dec64[0.005254895622179379, 0.02063885341193556, 0.002255248323355989, 0.01370885753833707, 0.007482408571946336, 0.0007481310045707236, 0.006093549383566077, 0.007787634107994545, 0.006687622148417179, 47.3505616147559  …  0.006360708202693763, 0.02221790165138363, 0.0006862305418845189, 0.001790634524407479, 0.006328177741941198, 0.0006137165469498114, 0.004800261873064844, 0.0030917103680649, 0.003494129413082352, 0.006343413907353043])

In [257]:
insertcols!(
    df_cand,
    :idx_match => idxs,
    :dist => dists,
);

In [258]:
columns = [
    :id => :p_all_id,
    :name => :name_p,
    :geonames_id => :geonames_id_p,
    :feature_class => :feature_class    
]

4-element Vector{Pair{Symbol, Symbol}}:
            :id => :p_all_id
          :name => :name_p
   :geonames_id => :geonames_id_p
 :feature_class => :feature_class

In [259]:
df_match = select(df_p_all[idxs, :], columns);

In [260]:
df_cand_p = hcat(df_cand, df_match);

In [261]:
df_no_match = subset(df_cand_p, :dist => ByRow(x -> x > 0.02));

In [262]:
columns = [    
    :geonamesId,
    :diocese,
    :placeName,
    :latitude,
    :longitude,
    :name_p,
    :geonames_id_p,    
];

In [263]:
select(df_no_match, columns)

Row,geonamesId,diocese,placeName,latitude,longitude,name_p,geonames_id_p
,String,String,String,Float64,Float64,String?,Int32?
1,2958914,Konstanz,Aichtal,48.6204,9.1715,Glashütte,2920107
2,2803175,Lüttich,Anhée,4.86042,50.3165,Leuca,2524399
3,6558305,Würzburg,Bad Rodach,50.3289,10.7597,Bad Rodach,2846166
4,6557834,Mainz,Frankenberg (Eder),51.1062,8.78296,Sachsenberg,2842556
5,6553285,Mainz,Haina (Kloster),51.0634,8.95611,Altenhaina,2957642
6,6557738,Mainz,Hanau,50.1333,8.99653,Rückingen,2843470
7,2748254,Utrecht,Reimerswaal,51.5002,4.16699,Strijenham,2746606
8,6544251,Utrecht,Schouwen-Duiveland,51.6704,3.87866,Kerkwerve,2752918
9,462322,Samland,Snamensk (Wehlau),54.614,21.2272,Melejdy,7535921


In [264]:
path_elmts = (
    "C:\\",
    "Users",
    "georg",
    "Documents",
    "projekte-doc",
    "WIAGweb2",
    "data",
    "Klosterort_P_check.csv"
)
out_file = joinpath(path_elmts)

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\Klosterort_P_check.csv"

In [265]:
CSV.write(out_file, df_no_match)

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\Klosterort_P_check.csv"

In [266]:
path_elmts = (
    "C:\\",
    "Users",
    "georg",
    "Documents",
    "projekte-doc",
    "WIAGweb2",
    "data",
    "Klosterort_P.csv"
)
out_file = joinpath(path_elmts)

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\Klosterort_P.csv"

In [268]:
CSV.write(out_file, df_cand_p)

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\Klosterort_P.csv"